# Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
Your homework will be divided into three tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

In [253]:
import pandas as pd
import requests
import bs4
import urllib.parse

# Ask for which city to pull retaurant stats from
restaurantCityState = input('What city/state/country would you like to pull? (e.g. "London, United Kingdom"): ')

# Format for appending to URL
restaurantCityState = urllib.parse.quote(restaurantCityState)

# Ask for number of rows of data to pull
restaurantCount = input('How many restaurants would you like to pull data for? Enter "all" to pull all restaurants. ')




What city/state/country would you like to pull? (e.g. "London, United Kingdom"): London, United Kingdom
How many restaurants would you like to pull data for? Enter "all" to pull all restaurants. 3


In [254]:
# Create function for pulling Yelp content based on the user-provided location and number of restaurants

def getYelpContent(restaurantCityState, restaurantCount):
    
    yelpURL = "https://www.yelp.com/search?find_desc=Restaurants&ns=1&find_loc=" + restaurantCityState
    
    loop = True
    grabAll = False
    rowPosition = 0
    content = []

    # If user entered "all" then set grabAll to true; if not, convert restaurantCount to int type
    if restaurantCount.lower() == "all":
        grabAll = True
    else:
        restaurantCount = int(restaurantCount)
    
    # Loop through Yelp pages while there's content available or until we reach/exceed 
    # the row count requested by user
    while loop:
        
        # Grab the rows (divs) of restaurant info from current page
        req = requests.get(yelpURL + "&start=" + str(rowPosition))
        scraper = bs4.BeautifulSoup(req.text)
        newRows = scraper.find_all('div', {'class': 'container__09f24__21w3G'})
        
        # If we scraped at least one row of restaurant info, add to our final content
        if len(newRows) > 0:
            content += newRows
            rowPosition += 10
        else:
            # No more new content to pull; exit loop
            loop = False
            
        # if the new rowPosition is more than what the user requested, exit loop
        if (not grabAll and rowPosition >= restaurantCount):
            loop = False
        
    # End while loop
    
    # Since the previous loop moves in increments of 10: if we grabbed too many rows, 
    # remove the excess rows (only if User didn't want to grab all rows)
    if not grabAll:
        while len(content) > restaurantCount:
            content.pop()
    
    return content
    
# End getYelpContent function


yelpRestaurants = getYelpContent(restaurantCityState, restaurantCount)



In [257]:
rest_name = []
rest_url = []
rest_priceRange = []
rest_rating = []
rest_reviewCount = []

# Loop through our list of restaurants; extract the data we want and append to the appopropiate list
for restaurant in yelpRestaurants:
    rest_name.append(restaurant.find_all('a', {'class': 'link-size--inherit__09f24__2Uj95'})[0].text)
    rest_url.append("https://www.yelp.com" + restaurant.find_all('a', {'class': 'link-size--inherit__09f24__2Uj95'})[0].attrs["href"])
    
    if len(restaurant.find_all('span', {'class': 'priceRange__09f24__2O6le'})) > 0:
        rest_priceRange.append(restaurant.find_all('span', {'class': 'priceRange__09f24__2O6le'})[0].text)
    else:
        rest_priceRange.append(None)

    if len(restaurant.find_all('div', {'class': 'i-stars__09f24__1T6rz'})) > 0:
        rest_rating.append(restaurant.find_all('div', {'class': 'i-stars__09f24__1T6rz'})[0].attrs["aria-label"].split()[0])
    else:
        rest_rating.append(None)

    if len(restaurant.find_all('span', {'class': 'reviewCount__09f24__EUXPN'})) > 0:
        rest_reviewCount.append(restaurant.find_all('span', {'class': 'reviewCount__09f24__EUXPN'})[0].text)
    else:
        rest_reviewCount.append(None)
        
    

In [258]:
# For the restaurant categories, we'll build a list of True/False values to specify 
# whether the restaurant is in a category

# Build a list of all possible restaurant categories by pulling from our list of restaurants
allPossibleCategories = []

for restaurant in yelpRestaurants:
    categoryHTML = restaurant.find_all('div', {'class': 'priceCategory__09f24__2IbAM'})
    categories = categoryHTML[0].find_all('a')
    
    for category in categories:
        allPossibleCategories.append(category.text)
    
# Sort and dedupe list of possible categories
allPossibleCategories = sorted(set(allPossibleCategories))

In [259]:
# Create our restaurant dictionary; categories will be added next

restaurant_dict = {
    'Name': rest_name,
    'URL': rest_url,
    'Price Range': rest_priceRange,
    'Rating': rest_rating,
    'Reviews': rest_reviewCount
}

In [260]:
# For each possible category, loop through our list of restaurants to check to see
# if the category is present for that restaurant

for possible_category in allPossibleCategories:
    
    this_category = []
    
    for restaurant in yelpRestaurants:
        
        categoryHTML = restaurant.find_all('div', {'class': 'priceCategory__09f24__2IbAM'})
        categories = categoryHTML[0].find_all('a')
        
        # Start by assuming the category is not present
        in_this_category = False
        
        for category in categories: 
            # if category is present, then turn to True
            if category.text == possible_category:
                in_this_category = True
        
        # Append restaurant's True/False to our results for this category
        this_category.append(in_this_category)
    
    # After checking all restaurants for this category, add the category to the dictionary
    restaurant_dict[possible_category] = this_category
            
    

In [261]:
# Convert dictionary to DataFrame

df = pd.DataFrame(restaurant_dict)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)

df

,Name,URL,Price Range,Rating,Reviews,British,Fish & Chips,French
0,The Mayfair Chippy,https://www.yelp.com/biz/the-mayfair-chippy-lo...,££,4.5,280,False,True,False
1,Ffiona’s Restaurant,https://www.yelp.com/biz/ffionas-restaurant-lo...,££,4.5,270,True,False,False
2,Restaurant Gordon Ramsay,https://www.yelp.com/biz/restaurant-gordon-ram...,££££,4.5,205,True,False,True
